# Building the network

The `nn.Module` subpackage in PyTorch contains many neural network building blocks called "modules". We can compose these in arbitrary ways to build network architectures tailored to a given problem. 

In [1]:
import torch
import torch.nn as nn

# do everything on gpu unless we explicitly say otherwise
torch.set_default_device('cuda')

## The basics

We saw examples like this in earlier notebooks: 

In [2]:
model = nn.Sequential(
    nn.Linear(10,10),
    nn.Tanh(),
    nn.Linear(10,10),
    nn.Tanh(),
    nn.Linear(10,3),
    nn.Sigmoid()
)

# printing the model shows the layers
model

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Tanh()
  (4): Linear(in_features=10, out_features=3, bias=True)
  (5): Sigmoid()
)

`nn.Sequential`, `nn.Linear`, `nn.Tanh`, and `nn.Sigmoid` are all examples of modules. There are many more. You can see a full list here: https://pytorch.org/docs/stable/nn.html

**Callable.** All modules are _callable_, meaning they can be evaluated like a function:

In [3]:
layer = nn.Linear(4,5)
x = torch.randn(7, 4)
layer(x)

tensor([[ 0.6022, -0.8395, -1.2719,  1.1310,  1.3735],
        [ 0.2362, -0.7734, -0.1520,  0.6548,  0.3336],
        [-1.1332,  0.3114, -0.7382, -0.3681,  1.1034],
        [ 0.6258, -0.9413,  1.1302, -0.0092,  0.2171],
        [-0.2644, -0.2822,  1.0074, -0.6441,  0.2830],
        [ 1.6555, -0.4360,  0.4600,  0.4437, -0.7630],
        [ 0.1289, -0.1957,  0.5850, -0.4486,  0.4141]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [4]:
layer = nn.Tanh()
layer(x)

tensor([[ 0.0532, -0.9158,  0.8895,  0.9291],
        [ 0.5709,  0.4384,  0.9173,  0.6719],
        [-0.9896, -0.5410, -0.6549,  0.7169],
        [ 0.9604,  0.3492,  0.3600, -0.3949],
        [ 0.1797,  0.4008, -0.6011, -0.5144],
        [ 0.9568, -0.3408,  0.9588, -0.9806],
        [ 0.0657, -0.4393, -0.4842, -0.6849]], device='cuda:0')

**Changing device.** Modules can be moved between devices. Unlike tensors, this operation is _in place_. 

In [5]:
layer = nn.Linear(4,5)
print("Before:", layer.weight.device)
layer.to('cpu')
print("After:", layer.weight.device)

Before: cuda:0
After: cpu


All nested modules also move: 

In [6]:
model = nn.Sequential(
    nn.Linear(10,10),
    nn.Tanh(),
    nn.Linear(10,3)
)

print("Before:", model[0].weight.device)
model.to('cpu')
print("After:", model[0].weight.device)

# back on gpu for later
model.to('cuda')

Before: cuda:0
After: cpu


Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=3, bias=True)
)

In [7]:
from utils import create_answer_box
create_answer_box("Recollection check! What's the purpose of the `tanh` layer in the above model? Why is it important to include this or something like it here?", "06-01")

Recollection check! What's the purpose of the `tanh` layer in the above model? Why is it important to include this or something like it here?

Text(value='', layout=Layout(width='500px'), placeholder='Type your answer here')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

**Saving/loading**. Model weights can be saved to and loaded from disc. There are a few ways to do this. The recommended way is to just save the weights using the "state dict" object:  

In [8]:
for k, v in model.state_dict().items():
    print(k, v.shape)

0.weight torch.Size([10, 10])
0.bias torch.Size([10])
2.weight torch.Size([3, 10])
2.bias torch.Size([3])


In [9]:
torch.save(model.state_dict(), 'model_weights.pt')

In [10]:
# Pytorch uses a version of pickle to save the weights
!head -n 3 model_weights.pt

PK                    model_weights/data.pklF ZZZZZZZZ�ccollections
OrderedDict
q )Rq(   0.weightqctorch._utils


In [13]:
# some time later...
model2 = nn.Sequential(
    nn.Linear(10,10),
    nn.Tanh(),
    nn.Linear(10,3)
)

model2.load_state_dict(torch.load('model_weights.pt', weights_only=True))

<All keys matched successfully>

Using the state dict required that we instantiate the model class first. We can also save the model structure together.

In [14]:
torch.save(model, 'model.pt')

In [16]:
model2 = torch.load('model.pt', weights_only=False)

Using `model.state_dict()` to save weights offers greater flexibility and compatibility, as it separates the model's parameters from its architecture, making it easy to update the model class or share weights. This approach results in smaller files and better portability across environments or versions of PyTorch, whereas saving the entire model (`torch.save(model, ...)`) is simpler but less adaptable to changes.

**`eval`/`train` modes.** Some layers need to behave differently at training time and evaluation time. These can all be toggled with the `train()` and `eval()` methods:

In [20]:
layer = nn.Dropout(0.5)

# the default mode is "training"
x = torch.randn(3, 5)
print(x)
layer(x)

tensor([[-0.3746, -0.2052,  0.1858, -1.1002, -0.8378],
        [ 0.5543,  0.1886,  0.5276,  0.2528,  0.9671],
        [ 0.8619,  0.5201,  1.9348,  0.0735,  0.8281]], device='cuda:0')


tensor([[-0.0000, -0.4104,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000,  1.0552,  0.0000,  1.9341],
        [ 1.7239,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0')

In [21]:
# switch to eval:
layer.eval()
layer(x)

tensor([[-0.3746, -0.2052,  0.1858, -1.1002, -0.8378],
        [ 0.5543,  0.1886,  0.5276,  0.2528,  0.9671],
        [ 0.8619,  0.5201,  1.9348,  0.0735,  0.8281]], device='cuda:0')

In [22]:
# switch back to train
layer.train()
layer(x)

tensor([[-0.0000, -0.0000, 0.3717, -0.0000, -0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.7239, 1.0403, 3.8697, 0.1469, 0.0000]], device='cuda:0')

## Writing custom modules

You can make your own modules. To do so, subclass `nn.Module` and define the `__init__` and `forward` method. These modules can be used just like any other module.

In [25]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        """
        The __init__ method defines all of the modules/parameters that will 
        appear in the model.
        """
        super().__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256,1)
        )

    def forward(self, x):
        """
        Define how to get from the input to the output. 
        You can use arbitrary python code here so long as the 
        tensor operations are differentiable. 
        """
        x = self.flatten(x)
        h = self.encoder(x)
        y = self.classifier(h)
        return y
    
model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [23]:
create_answer_box("The `forward` method is very flexible, but there are limits. Can you think of an example of code that would NOT work in the `forward` method? Why would it not work?", "06-02")

The `forward` method is very flexible, but there are limits. Can you think of an example of code that would NOT work in the `forward` method? Why would it not work?

Text(value='', layout=Layout(width='500px'), placeholder='Type your answer here')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

In [26]:
# simulate a batch of grayscale images:
x = torch.randn(5, 1, 28, 28)

model(x)

tensor([[0.0784],
        [0.0576],
        [0.1319],
        [0.1175],
        [0.0928]], device='cuda:0', grad_fn=<AddmmBackward0>)

You can customize your network however you see fit. For example, say we had a problem where the network took two images as input and made some decision about them. We could do something like this: 

In [27]:
class PairNetwork(nn.Module):
    def __init__(self):
        """
        The __init__ method defines all of the modules/parameters that will 
        appear in the model.
        """
        super().__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(2*256,1)  # double the representation size
        )

    def forward(self, x1, x2):
        """
        Define how to get from the input to the output. 
        You can use arbitrary python code here so long as the 
        tensor operations are differentiable. 
        """
        x1 = self.flatten(x1)
        h1 = self.encoder(x1)
        
        x2 = self.flatten(x2)
        h2 = self.encoder(x2)
        
        # fuse the representations
        h = torch.concat([h1, h2], axis=-1)
        
        y = self.classifier(h)
        return y
    
pair_model = PairNetwork()
pair_model

PairNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [28]:
# simulate a batch of grayscale images:
x1 = torch.randn(5, 1, 28, 28)
x2 = torch.randn(5, 1, 28, 28)

pair_model(x1, x2)

tensor([[ 0.0376],
        [-0.0259],
        [-0.0776],
        [-0.0457],
        [-0.0491]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [29]:
create_answer_box("This network takes two images and concatenates their representations. Can you think of real-world applications where you'd want to compare or combine multiple inputs like this?", "06-03")

This network takes two images and concatenates their representations. Can you think of real-world applications where you'd want to compare or combine multiple inputs like this?

Text(value='', layout=Layout(width='500px'), placeholder='Type your answer here')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

**Tracking parameters** Pytorch automatically tracks all of the parameters that appear in your custom model. This allows Pytorch to optimize the network during training. It allows can allow you to get diagnostic information such as the number of parameters in your model: 

In [30]:
num_pars = sum([p.numel() for p in model.parameters()])
print("Number of parameters:", num_pars)

Number of parameters: 796161
